In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
drive_project_root = 'drive/MyDrive/ColabNotebooks/'

In [ ]:
pip install pytorch-crf

In [ ]:
pip install gensim==3.4.0

In [ ]:
pip install sentencepiece

In [ ]:
import os
import sys
import json
import random
import pandas as pd
import numpy as np
from collections import deque

from drive.MyDrive.ColabNotebooks.src.model import BiLSTM_CRF, MakeEmbed, textCNN, EpochLogger, save
from drive.MyDrive.ColabNotebooks.src.dataset import Preprocessing, MakeDataset
from drive.MyDrive.ColabNotebooks.src.NLU import NaturalLanguageUnderstanding
from drive.MyDrive.ColabNotebooks.src.DM import DialogManager, E2E_dialog

In [ ]:
class DialogSystem:
    def __init__(self):
        intent_pretrain_path = drive_project_root+"data/pretraining/save/1_intent_clsf_model/intent_clsf_97.217_steps_33.pt"
        entity_pretrain_path = drive_project_root+"data/pretraining/save/1_entity_recog_model/entity_recog_97.192_steps_7.pt"
        ood_pretrain_path = drive_project_root+"data/pretraining/save/1_ood_clsf_model/ood_clsf_99.724_steps_5.pt"
        chitchat_pretrain_path = drive_project_root+"data/pretraining/save/4_chitchat_transformer_model/chitchat_transformer_1.215381_steps_81.pt"

        self.NLU = NaturalLanguageUnderstanding()

        self.NLU.model_load(intent_pretrain_path, entity_pretrain_path, ood_pretrain_path)
        self.dm = DialogManager()

        self.e2e = E2E_dialog(self.NLU.dataset,chitchat_pretrain_path)
        self.result = {
            "DM_result": {},    # 목적지향 대화시스템 결과
            "E2E_result" : [],  # 오픈도메인 대화시스템 결과
            "NLG"  : "",        # 최종 선택된 것
        }
    def task_oriented_dialog_system(self, q):
        NLU_result = self.NLU.run(q)

        system_response = self.dm.run(NLU_result)
        return system_response
    
    def chit_chat_dialog_system(self, q):
        system_response = self.e2e.predict(q)
        return system_response
    
    # 내 방식대로 바꿔도 됨
    def ranking(self, dm_result, chitchat_result):
        self.result["DM_result"] = dm_result
        self.result["E2E_result"] = list(chitchat_result)
        dm_nlg = dm_result['NLG']
        
        # 목적지향 대화시스템 결과를 우선시함
        # 목적지향 대화시스템이 응답을 할 수 있는 거면 분명한 의도가 있다고 판단했기 때문
        if(not dm_nlg == []):   # 목적지향 대화시스템 결과가 있으면 바로 그 결과 출력
            self.result["NLG"] = dm_nlg
            return self.result
        else:                   # 없으면 오픈도메인 대화시스템의 문장 생성 확률을 보고 
            score = chitchat_result[1]
            print(score)
            if(score>0.05):     # 확률이 0.05보다 크면 오픈도메인 대화시스템 결과 출력
                self.result["NLG"] = chitchat_result[0]
                return self.result
            else:
                self.result["NLG"] = "죄송합니다. 다시 말해주세요."
                return self.result
            
    def run(self, q):
        dm_result = self.task_oriented_dialog_system(q)    # 목적지향 대화시스템 결과
        chitchat_result = self.chit_chat_dialog_system(q)  # e2e 대화시스템 결과
        result = self.ranking(dm_result, chitchat_result)  # ranking
        return result

In [ ]:
dialogsystem = DialogSystem()

In [ ]:
result=dialogsystem.run("안녕")

In [ ]:
result

In [ ]:
result=dialogsystem.run("인천 맛집")

In [ ]:
result

In [ ]:
result=dialogsystem.run("미세먼지 알려줘")

In [ ]:
result

In [ ]:
result=dialogsystem.run("대전")

In [ ]:
result

In [ ]:
dialogsystem.run("날씨 알려줘")

In [ ]:
result=dialogsystem.run("난 혼자인게 좋아")

"난 혼자인게 좋아"에 대한 결과
- 목적지향 대화시스템 : 응답 불가
- 오픈도메인 대화시스템 : "이제 일어날 시간이에요 놓아주세요" -> 0.25라 0.05보다 커서 이 말이 출력될 듯

In [ ]:
result

In [ ]:
result=dialogsystem.run("꽃을 좋아해")

In [ ]:
result

### 실제 대화 해보기
- 종료 : end

In [ ]:
dialogsystem.dm.clear()

q = ""
q = input()
while(q!="end"):
    
    result=dialogsystem.run(q)
    print(result['NLG'])
    q = input()
    
dialogsystem.dm.clear()